<a href="https://colab.research.google.com/github/ANGJARG1734/FOLD4/blob/main/K_FOLD_FOLD_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas ultralytics pyyaml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 20.0 MB/s eta 0:00:00


In [3]:
import torch
torch.cpu.is_available()

True

In [4]:
!nvidia-smi

Fri Oct 25 16:12:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
#Este archivo verifica que las imágenes tengan su etiqueta en su folder correspondiente

import os
from pathlib import Path

# Función para verificar que cada imagen tenga su etiqueta correspondiente
def verify_image_label_pairs_and_count(split_folder):
    image_extensions = ['.jpg', '.jpeg', '.png']  # Extensiones soportadas
    total_images = 0
    total_labels = 0

    # Iterar sobre las carpetas 'train' y 'val' dentro del split
    for folder_type in ['train', 'val']:
        images_path = split_folder / folder_type
        print(f"\nVerificando en carpeta: {images_path}")

        image_count = 0
        missing_pairs = []

        # Verificar imágenes con múltiples extensiones
        for ext in image_extensions:
            for image_file in images_path.glob(f"*{ext}"):  # Buscar imágenes con cualquier extensión
                label_file = images_path / f"{image_file.stem}.txt"  # Buscar archivo .txt correspondiente
                #print("Checando archivos: ")
                #print(label_file)
                #print(image_file)
                if not label_file.exists():
                    missing_pairs.append(image_file.name)  # Añadir a lista de faltantes si no tiene etiqueta
                else:
                    image_count += 1

        # Imprimir el resultado de la verificación
        if missing_pairs:
            print(f"Imágenes sin etiqueta en {folder_type}:")
            for missing in missing_pairs:
                print(missing)
        else:
            print(f"Todas las imágenes en {folder_type} tienen su etiqueta correspondiente.")

        # Contar el número total de imágenes y etiquetas en esta carpeta
        print(f"Número de imágenes con etiquetas en {folder_type}: {image_count}")

        # Sumar a los contadores totales
        total_images += image_count
        total_labels += image_count  # Porque cada imagen con etiqueta tiene un txt

    # Retornar el total de imágenes y etiquetas en todo el split
    return total_images, total_labels

# Función principal para verificar todos los splits
def verify_splits(splits_folder):
    # Iterar sobre los splits (del 1 al 5)
    for i in range(1, 6):
        split_folder = splits_folder / f"split_{i}"
        print(f"\nVerificando Split {i}")

        # Verificar pares y contar archivos en el split actual
        total_images, total_labels = verify_image_label_pairs_and_count(split_folder)

        # Calcular y mostrar la cantidad de imágenes
        if total_images == total_labels:
            print(f"Split {i} tiene un total de {total_images} imágenes (todas con su etiqueta correspondiente).")
        else:
            print(f"Alerta en Split {i}: imágenes y etiquetas no coinciden.")

# Ruta principal donde están los splits (ya generados previamente)
splits_path = Path(r"/content/drive/MyDrive/YOLO/k_folds/Project1")

# Verificar todos los splits
verify_splits(splits_path)



Verificando Split 1

Verificando en carpeta: /content/drive/MyDrive/YOLO/k_folds/Project1/split_1/train
Todas las imágenes en train tienen su etiqueta correspondiente.
Número de imágenes con etiquetas en train: 414

Verificando en carpeta: /content/drive/MyDrive/YOLO/k_folds/Project1/split_1/val
Todas las imágenes en val tienen su etiqueta correspondiente.
Número de imágenes con etiquetas en val: 104
Split 1 tiene un total de 518 imágenes (todas con su etiqueta correspondiente).

Verificando Split 2

Verificando en carpeta: /content/drive/MyDrive/YOLO/k_folds/Project1/split_2/train
Todas las imágenes en train tienen su etiqueta correspondiente.
Número de imágenes con etiquetas en train: 414

Verificando en carpeta: /content/drive/MyDrive/YOLO/k_folds/Project1/split_2/val
Todas las imágenes en val tienen su etiqueta correspondiente.
Número de imágenes con etiquetas en val: 104
Split 2 tiene un total de 518 imágenes (todas con su etiqueta correspondiente).

Verificando Split 3

Verifica

In [6]:
import os
from pathlib import Path

# Función para verificar que no haya duplicados en las carpetas de validación de cada split
def verify_unique_validation_files(splits_folder):
    val_files_set = set()  # Para almacenar los nombres únicos de los archivos de validación
    duplicates = []  # Para almacenar los archivos duplicados encontrados
    all_val_files = {}  # Diccionario para almacenar en qué split está cada archivo

    # Iterar sobre los splits (del 1 al 5)
    for i in range(1, 6):
        split_folder = splits_folder / f"split_{i}" / 'val'
        print(f"\nEscaneando carpeta de validación en Split {i}: {split_folder}")

        # Verificar imágenes con múltiples extensiones
        for ext in ['.jpg', '.jpeg', '.png']:
            for image_file in split_folder.glob(f"*{ext}"):
                #print("Archivos:")
                #print(image_file)
                if image_file.name in val_files_set:
                    # Si ya existe, es un duplicado
                    duplicates.append(image_file.name)
                    print(f"Duplicado encontrado: {image_file.name} ya estaba en {all_val_files[image_file.name]}")
                else:
                    # Si no existe, lo agregamos al set y guardamos el split donde apareció
                    val_files_set.add(image_file.name)
                    all_val_files[image_file.name] = f"Split {i}"

    # Mostrar resultados
    if duplicates:
        print("\nSe encontraron los siguientes archivos duplicados en las carpetas de validación:")
        for dup in duplicates:
            print(dup)
    else:
        print("\nNo se encontraron archivos duplicados en las carpetas de validación entre los splits.")
        print("Folders de validación únicos")

# Ruta principal donde están los splits
splits_path = Path(r"C:\Users\guza_\OneDrive\Documentos\ITSZ\Semestres\Residencia\Develop\Python\k_fold_versions\version2\Project\Dataset\2024-10-24_5-Fold_Cross-val")

# Verificar unicidad de archivos en las carpetas de validación
verify_unique_validation_files(splits_path)



Escaneando carpeta de validación en Split 1: C:\Users\guza_\OneDrive\Documentos\ITSZ\Semestres\Residencia\Develop\Python\k_fold_versions\version2\Project\Dataset\2024-10-24_5-Fold_Cross-val/split_1/val

Escaneando carpeta de validación en Split 2: C:\Users\guza_\OneDrive\Documentos\ITSZ\Semestres\Residencia\Develop\Python\k_fold_versions\version2\Project\Dataset\2024-10-24_5-Fold_Cross-val/split_2/val

Escaneando carpeta de validación en Split 3: C:\Users\guza_\OneDrive\Documentos\ITSZ\Semestres\Residencia\Develop\Python\k_fold_versions\version2\Project\Dataset\2024-10-24_5-Fold_Cross-val/split_3/val

Escaneando carpeta de validación en Split 4: C:\Users\guza_\OneDrive\Documentos\ITSZ\Semestres\Residencia\Develop\Python\k_fold_versions\version2\Project\Dataset\2024-10-24_5-Fold_Cross-val/split_4/val

Escaneando carpeta de validación en Split 5: C:\Users\guza_\OneDrive\Documentos\ITSZ\Semestres\Residencia\Develop\Python\k_fold_versions\version2\Project\Dataset\2024-10-24_5-Fold_Cross-v

In [7]:
# Importar la librería ultralytics
from ultralytics import YOLO
import os

# Desactivar Weights & Biases (W&B)
os.environ["WANDB_MODE"] = "disabled"


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [8]:
!yolo task=detect mode=train epochs=90 data=/content/drive/MyDrive/YOLO/k_folds/Project1/split_5/split_5_dataset.yaml model=yolov8m.pt imgsz=640 batch=8 project="fold_5_run" name="siu_5_run" verbose=True amp=False


100% 49.7M/49.7M [00:00<00:00, 334MB/s]
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/YOLO/k_folds/Project1/split_5/split_5_dataset.yaml, epochs=90, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=fold_5_run, name=siu_5_run, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals